# Python3 网络爬虫开发实战 | 第 三 章_基本库的使用 (1)

站在巨人的肩膀上学习，感谢崔庆才大神，本系列仅记录学习笔记及个人理解，若有错误，还望批评指教。——ZJ

>原书作者：崔庆才 ，网站 https://cuiqingcai.com/ | https://cuiqingcai.com/5052.html  Github : https://github.com/Germey


---


# 3-基本库的使用

## 3.1-使用 urllib

使用 urllib 的 request 模块，可以方便地实现请求的发送并得到响应。

### 1. urlopen()

urllib.request 模块提供了最基本的构造 HTTP 请求的方法，利用它可以模拟浏览器的一个请求发起过程，同时它还带有处理授权验证（authenticaton）、重定向（redirection)、浏览器 Cookies 以及其他内容。


In [1]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')

print(response.read().decode('utf-8'))

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="HandheldFriendly" conte

In [2]:
import urllib.request

response = urllib.request.urlopen('https://www.python.org')
print(type(response))

<class 'http.client.HTTPResponse'>


可以发现，它是一个HTTPResposne类型的对象。它主要包含`read()、readinto()、getheader(name)、getheaders()、fileno()`等方法，以及`msg、version、status、reason、debuglevel、closed `等属性。

In [6]:
print("status",response.status)
print("getheaders()",response.getheaders())
print("getheader(Server)",response.getheader('Server'))

status 200
getheaders() [('Server', 'nginx'), ('Content-Type', 'text/html; charset=utf-8'), ('X-Frame-Options', 'SAMEORIGIN'), ('x-xss-protection', '1; mode=block'), ('X-Clacks-Overhead', 'GNU Terry Pratchett'), ('Via', '1.1 varnish'), ('Content-Length', '48752'), ('Accept-Ranges', 'bytes'), ('Date', 'Tue, 15 May 2018 00:25:58 GMT'), ('Via', '1.1 varnish'), ('Age', '3407'), ('Connection', 'close'), ('X-Served-By', 'cache-iad2123-IAD, cache-nrt6143-NRT'), ('X-Cache', 'HIT, HIT'), ('X-Cache-Hits', '1, 12'), ('X-Timer', 'S1526343958.170475,VS0,VE3'), ('Vary', 'Cookie'), ('Strict-Transport-Security', 'max-age=63072000; includeSubDomains')]
getheader(Server) nginx


可见，前两个输出分别输出了响应的状态码和响应的头信息，最后一个输出通过调用 `getheader()`方法并传递一个参数`Server`获取了响应头中的Server值，结果是nginx，意思是服务器是用 Nginx 搭建的。

**urlopen()函数的 API:**

```
urllib.request.urlopen(url, data=None, [timeout, ]*, cafile=None, capath=None, cadefault=False, context=None)

```

- data 参数

`data`参数是可选的。如果要添加该参数，并且如果它是字节流编码格式的内容，即`bytes`类型，则需要通过`bytes()`方法转化。另外，如果传递了这个参数，则它的请求方式就不再是 GET 方式，而是 POST 方式。

In [7]:
import urllib.parse
import urllib.request

data = bytes(urllib.parse.urlencode({'word':'hello'}), encoding='utf-8')

response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read())

b'{"args":{},"data":"","files":{},"form":{"word":"hello"},"headers":{"Accept-Encoding":"identity","Connection":"close","Content-Length":"10","Content-Type":"application/x-www-form-urlencoded","Host":"httpbin.org","User-Agent":"Python-urllib/3.6"},"json":null,"origin":"124.207.145.162","url":"http://httpbin.org/post"}\n'


这里我们传递了一个参数word，值是hello。它需要被转码成bytes（字节流）类型。其中转字节流采用了bytes()方法，该方法的第一个参数需要是str（字符串）类型，需要用urllib.parse模块里的urlencode()方法来将参数字典转化为字符串；第二个参数指定编码格式，这里指定为utf8。

这里请求的站点是httpbin.org，它可以提供HTTP请求测试。本次我们请求的URL为 http://httpbin.org/post ， 这个链接可以用来测试POST请求，它可以输出请求的一些信息，其中包含我们传递的data参数。

我们传递的参数出现在了 form 字段中，这表明是模拟了表单提交的方式，以POST 方式传输数据。


**timeout参数**

timeout参数用于设置超时时间，单位为秒，意思就是如果请求超出了设置的这个时间，还没有得到响应，就会抛出异常。如果不指定该参数，就会使用全局默认时间。它支持HTTP、HTTPS、FTP请求。

In [8]:
response = urllib.request.urlopen('http://httpbin.org/get', timeout=1)
print(response.read())

timeout: timed out

In [9]:
import socket
import urllib.request
import urllib.error

try:
    response = urllib.request.urlopen('http://httpbin.org/get', timeout=0.1)
except urllib.error.URLError as e:
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')

TIME OUT


### 2. Request

我们知道利用urlopen()方法可以实现最基本请求的发起，但这几个简单的参数并不足以构建一个完整的请求。如果请求中需要加入Headers等信息，就可以利用更强大的Request类来构建。

首先，我们用实例来感受一下Request的用法：


In [11]:
import urllib.request

request = urllib.request.Request('https://python.org')
response = urllib.request.urlopen(request)
print(response.read().decode('utf-8'))

                                 

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="HandheldFriendly" conte

Request可以通过怎样的参数来构造，它的构造方法如下:

```
	
class urllib.request.Request(url, data=None, headers={}, origin_req_host=None, unverifiable=False, method=None)
```

- 第一个参数url用于请求URL，这是必传参数，其他都是可选参数。
- 第二个参数data如果要传，必须传bytes（字节流）类型的。如果它是字典，可以先用urllib.parse模块里的urlencode()编码。
- 第三个参数headers是一个字典，它就是请求头，我们可以在构造请求时通过headers参数直接构造，也可以通过调用请求实例的add_header()方法添加。
添加请求头最常用的用法就是通过修改User-Agent来伪装浏览器，默认的User-Agent是Python-urllib，我们可以通过修改它来伪装浏览器。比如要伪装火狐浏览器，你可以把它设置为：

`Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)`

- 第四个参数origin_req_host指的是请求方的host名称或者IP地址。
- 第五个参数unverifiable表示这个请求是否是无法验证的，默认是False，意思就是说用户没有足够权限来选择接收这个请求的结果。例如，我们请求一个HTML文档中的图片，但是我们没有自动抓取图像的权限，这时`unverifiable` 的值就是 `True`。
- 第六个参数method是一个字符串，用来指示请求使用的方法，比如GET、POST和PUT等。

下面我们传入多个参数构建请求来看一下：


In [13]:
from urllib import request, parse

url = 'http://httpbin.org/post'

headers = {
    'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    'Host':'httpbin.org'    
}

dict1 = {
    'name':'Germey'
}

data = bytes(parse.urlencode(dict1), encoding='utf-8')

req = request.Request(url=url, data=data, headers=headers, method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{"args":{},"data":"","files":{},"form":{"name":"Germey"},"headers":{"Accept-Encoding":"identity","Connection":"close","Content-Length":"11","Content-Type":"application/x-www-form-urlencoded","Host":"httpbin.org","User-Agent":"Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)"},"json":null,"origin":"124.207.145.162","url":"http://httpbin.org/post"}



### 3. 高级用法


https://cuiqingcai.com/5500.html

在上面的过程中，我们虽然可以构造请求，但是对于一些更高级的操作（比如Cookies处理、代理设置等），我们该怎么办呢？

接下来，就需要更强大的工具Handler登场了。简而言之，我们可以把它理解为各种处理器，有专门处理登录验证的，有处理Cookies的，有处理代理设置的。利用它们，我们几乎可以做到HTTP请求中所有的事情。

首先，介绍一下urllib.request模块里的BaseHandler类，它是所有其他Handler的父类，它提供了最基本的方法，例如default_open()、protocol_request()等。

接下来，就有各种Handler子类继承这个BaseHandler类，举例如下。

- HTTPDefaultErrorHandler：用于处理HTTP响应错误，错误都会抛 HTTPError类型的异常。
- HTTPRedirectHandler：用于处理重定向。
- HTTPCookieProcessor：用于处理Cookies。
- ProxyHandler：用于设置代理，默认代理为空。
- HTTPPasswordMgr：用于管理密码，它维护了用户名和密码的表。
- HTTPBasicAuthHandler：用于管理认证，如果一个链接打开时需要认证，那么可以用它来解决认证问题。

另一个比较重要的类就是OpenerDirector，我们可以称为Opener。我们之前用过urlopen()这个方法，实际上它就是urllib为我们提供的一个Opener。

那么，为什么要引入Opener呢？因为需要实现更高级的功能。之前使用的Request和urlopen()相当于类库为你封装好了极其常用的请求方法，利用它们可以完成基本的请求，但是现在不一样了，我们需要实现更高级的功能，所以需要深入一层进行配置，使用更底层的实例来完成操作，所以这里就用到了Opener。

Opener可以使用open()方法，返回的类型和urlopen()如出一辙。那么，它和Handler有什么关系呢？简而言之，就是利用Handler来构建Opener。

下面用几个实例来看看它们的用法。

**验证**

有些网站在打开时就会弹出提示框，直接提示你输入用户名和密码，验证成功后才能查看页面，如图3-2所示。

In [14]:
from urllib.request import HTTPPasswordMgrWithDefaultRealm, HTTPBasicAuthHandler,build_opener

from urllib.error import URLError

username = 'username'
password = 'password'
url = 'http://localhost:5000/'

p = HTTPPasswordMgrWithDefaultRealm()
p.add_password(None, url, username, password)
auth_handler = HTTPBasicAuthHandler(p)
opener = build_opener(auth_handler)

try:
    result = opener.open(url)
    html = result.read().decode('utf-8')
    print(html)
except URLError as e:
    print(e.reason)

[WinError 10061] 由于目标计算机积极拒绝，无法连接。


这里首先实例化HTTPBasicAuthHandler对象，其参数是HTTPPasswordMgrWithDefaultRealm对象，它利用add_password()添加进去用户名和密码，这样就建立了一个处理验证的Handler。

接下来，利用这个Handler并使用build_opener()方法构建一个Opener，这个Opener在发送请求时就相当于已经验证成功了。

接下来，利用Opener的open()方法打开链接，就可以完成验证了。这里获取到的结果就是验证后的页面源码内容。


**代理**

在做爬虫的时候，免不了要使用代理，如果要添加代理，可以这样做：

In [16]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy_handler = ProxyHandler({
    'http':'http://127.0.0.1:9743',
    'https':'https:127.0.0.1:9743'
})
opener = build_opener(proxy_handler)

try:
    response = opener.open('https://www.baidu.com')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

[Errno 11001] getaddrinfo failed


里我们在本地搭建了一个代理，它运行在9743端口上。

这里使用了ProxyHandler，其参数是一个字典，键名是协议类型（比如HTTP或者HTTPS等），键值是代理链接，可以添加多个代理。

然后，利用这个Handler及build_opener()方法构造一个Opener，之后发送请求即可。

**Cookies**

Cookies的处理就需要相关的Handler了。

我们先用实例来看看怎样将网站的Cookies获取下来，相关代码如下

In [17]:
import http.cookiejar, urllib.request 

cookie = http.cookiejar.CookieJar()
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
for item in cookie:
    print(item.name+"="+item.value)

BAIDUID=0641FE2ED330C8520C7F1C14FE4FB8C7:FG=1
BIDUPSID=0641FE2ED330C8520C7F1C14FE4FB8C7
H_PS_PSSID=1445_21106_26350
PSTM=1526349089
BDSVRTM=0
BD_HOME=0


In [18]:
filename = 'cookies.txt'
cookie = http.cookiejar.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard=True, ignore_expires=True)

In [ ]:
# Netscape HTTP Cookie File
# http://curl.haxx.se/rfc/cookie_spec.html
# This is a generated file!  Do not edit.

.baidu.com	TRUE	/	FALSE	3673833047	BAIDUID	34F40DEFC2E43C80EE29D00C418EA516:FG=1
.baidu.com	TRUE	/	FALSE	3673833047	BIDUPSID	34F40DEFC2E43C80EE29D00C418EA516
.baidu.com	TRUE	/	FALSE		H_PS_PSSID	1422_21099_20927
.baidu.com	TRUE	/	FALSE	3673833047	PSTM	1526349395
www.baidu.com	FALSE	/	FALSE		BDSVRTM	0
www.baidu.com	FALSE	/	FALSE		BD_HOME	0


In [19]:
filename = 'cookies1.txt'
cookie = http.cookiejar.LWPCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard=True, ignore_expires=True)

In [ ]:
#LWP-Cookies-2.0
Set-Cookie3: BAIDUID="E7C68B09D65ACD5A3EC8E5318346C4A2:FG=1"; path="/"; domain=".baidu.com"; path_spec; domain_dot; expires="2086-06-02 05:12:52Z"; version=0
Set-Cookie3: BIDUPSID=E7C68B09D65ACD5A3EC8E5318346C4A2; path="/"; domain=".baidu.com"; path_spec; domain_dot; expires="2086-06-02 05:12:52Z"; version=0
Set-Cookie3: H_PS_PSSID=26374_1465_21084_26350_20929; path="/"; domain=".baidu.com"; path_spec; domain_dot; discard; version=0
Set-Cookie3: PSTM=1526349520; path="/"; domain=".baidu.com"; path_spec; domain_dot; expires="2086-06-02 05:12:52Z"; version=0
Set-Cookie3: BDSVRTM=0; path="/"; domain="www.baidu.com"; path_spec; discard; version=0
Set-Cookie3: BD_HOME=0; path="/"; domain="www.baidu.com"; path_spec; discard; version=0


In [20]:
cookie = http.cookiejar.LWPCookieJar()
cookie.load('cookies1.txt', ignore_discard=True, ignore_expires=True)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
print(response.read().decode('utf-8'))

<!DOCTYPE html>
<!--STATUS OK-->
























































	
































	
        
			        
	
			        
	
			        
	
			        
			    

	
        
			        
	
			        
	
			        
	
			        
			    
























<html>
<head>
    
    <meta http-equiv="content-type" content="text/html;charset=utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta content="always" name="referrer">
    <meta name="theme-color" content="#2932e1">
    <link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" />
    <link rel="search" type="application/opensearchdescription+xml" href="/content-search.xml" title="百度搜索" />
    <link rel="icon" sizes="any" mask href="//www.baidu.com/img/baidu_85beaf5496f291521eb75ba38eacbd87.svg">
	
	
	<link rel="dns-prefetch" href="//s1.bdstatic.com"/>
	<link rel="dns-prefetch" href="//t1.baidu.com"/>
	<link rel="dns-prefetch" href="//t2.baidu.com"/>
	<link rel="dns-pre

可以看到，这里调用load()方法来读取本地的Cookies文件，获取到了Cookies的内容。不过前提是我们首先生成了LWPCookieJar格式的Cookies，并保存成文件，然后读取Cookies之后使用同样的方法构建Handler和Opener即可完成操作。

运行结果正常的话，会输出百度网页的源代码。

通过上面的方法，我们可以实现绝大多数请求功能的设置了。

这便是urllib库中request模块的基本用法，如果想实现更多的功能，可以参考官方文档的说明：https://docs.python.org/3/library/urllib.request.html#basehandler-objects。

## 3.1.2-处理异常

urllib的error模块定义了由request模块产生的异常。如果出现了问题，request模块便会抛出error模块中定义的异常。

### 1. URLError

URLError类来自urllib库的error模块，它继承自OSError类，是error异常模块的基类，由request模块生的异常都可以通过捕获这个类来处理。

它具有一个属性reason，即返回错误的原因。

In [21]:
from urllib import request, error

try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.URLError as e:
    print(e.reason)

Not Found


### 2. HTTPError

它是URLError的子类，专门用来处理HTTP请求错误，比如认证请求失败等。它有如下3个属性。

- code：返回HTTP状态码，比如404表示网页不存在，500表示服务器内部错误等。
- reason：同父类一样，用于返回错误的原因。
- headers：返回请求头。



In [22]:
from urllib import request, error
try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.HTTPError  as e:
    print(e.reason, e.code, e.headers, sep='\n')

Not Found
404
Server: nginx/1.10.3 (Ubuntu)
Date: Tue, 15 May 2018 02:12:44 GMT
Content-Type: text/html; charset=UTF-8
Transfer-Encoding: chunked
Connection: close
Vary: Cookie
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Cache-Control: no-cache, must-revalidate, max-age=0
Link: <https://cuiqingcai.com/wp-json/>; rel="https://api.w.org/"




In [23]:
from urllib import request, error

try:
    response = request.urlopen('http://cuiqingcai.com/index.htm')
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')
except error.URLError as e:
    print(e.reason)
else:
    print('Request Successfully')
    

Not Found
404
Server: nginx/1.10.3 (Ubuntu)
Date: Tue, 15 May 2018 02:16:29 GMT
Content-Type: text/html; charset=UTF-8
Transfer-Encoding: chunked
Connection: close
Vary: Cookie
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Cache-Control: no-cache, must-revalidate, max-age=0
Link: <https://cuiqingcai.com/wp-json/>; rel="https://api.w.org/"




In [24]:
import socket
import urllib.request
import urllib.error

try:
    response = urllib.request.urlopen('https://www.baidu.com', timeout=0.01)
except urllib.error.URLError as e:
    print(type(e.reason))
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')


<class 'socket.timeout'>
TIME OUT


可以发现，reason属性的结果是socket.timeout类。所以，这里我们可以用isinstance()方法来判断它的类型，作出更详细的异常判断。

## 3.1.3-解析链接

https://cuiqingcai.com/5508.html

前面说过，urllib库里还提供了parse这个模块，它定义了处理URL的标准接口，例如实现URL各部分的抽取、合并以及链接转换。它支持如下协议的URL处理：file、ftp、gopher、hdl、http、https、imap、mailto、 mms、news、nntp、prospero、rsync、rtsp、rtspu、sftp、 sip、sips、snews、svn、svn+ssh、telnet和wais。本节中，我们介绍一下该模块中常用的方法来看一下它的便捷之处。

### 1. urlparse()

该方法可以实现URL的识别和分段，这里先用一个实例来看一下：

In [25]:
from urllib.parse import urlparse

result = urlparse('http://www.baidu.com/index.html;user?id=5#comment')

print(type(result), result)

<class 'urllib.parse.ParseResult'> ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


所以，可以得出一个标准的链接格式，具体如下：

`scheme://netloc/path;parameters?query#fragment`



In [26]:
from urllib.parse import urlparse

result = urlparse('www.baidu.com/index.html;user?id=5#comment', scheme='https')
print(result)

ParseResult(scheme='https', netloc='', path='www.baidu.com/index.html', params='user', query='id=5', fragment='comment')


In [28]:
result = urlparse('http://www.baidu.com/index.html;user?id=5#comment', scheme='https')
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


可见，scheme参数只有在URL中不包含scheme信息时才生效。如果URL中有scheme信息，就会返回解析出的scheme。

allow_fragments：即是否忽略fragment。如果它被设置为False，fragment部分就会被忽略，它会被解析为path、parameters或者query的一部分，而fragment部分为空。下面我们用实例来看一下：

In [29]:
from urllib.parse import urlparse

result = urlparse('http://www.baidu.com/index.html;user?id=5#comment', allow_fragments=False)
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5#comment', fragment='')


In [30]:
result = urlparse('http://www.baidu.com/index.html#comment', allow_fragments=False)
print(result)

ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html#comment', params='', query='', fragment='')


可以发现，当URL中不包含params和query时，fragment便会被解析为path的一部分。

返回结果ParseResult实际上是一个元组，我们可以用索引顺序来获取，也可以用属性名获取。示例如下：

In [31]:
from urllib.parse import urlparse

result = urlparse('http://www.baidu.com/index.html#comment', allow_fragments=False)
print(result.scheme, result[0], result.netloc, result[1], sep='\n')

http
http
www.baidu.com
www.baidu.com


### 2. urlunparse()

有了urlparse()，相应地就有了它的对立方法urlunparse()。它接受的参数是一个可迭代对象，但是它的长度必须是6，否则会抛出参数数量不足或者过多的问题。先用一个实例看一下：

In [33]:
from urllib.parse import urlunparse

data = ['http', 'www.baidu.com', 'index.html', 'user', 'a=6', 'comment']
print(urlunparse(data))

http://www.baidu.com/index.html;user?a=6#comment


### 3. urlsplit()

这个方法和urlparse()方法非常相似，只不过它不再单独解析params这一部分，只返回5个结果。上面例子中的params会合并到path中。示例如下

In [34]:
from urllib.parse import urlsplit

result = urlsplit('http://www.baidu.com/index.html;user?id=5#comment')
print(result)

SplitResult(scheme='http', netloc='www.baidu.com', path='/index.html;user', query='id=5', fragment='comment')


In [35]:
from urllib.parse import urlsplit

result = urlsplit('http://www.baidu.com/index.html;user?id=5#comment')
print(result.scheme, result[0])

http http


### 4. urlunsplit()

与urlunparse()类似，它也是将链接各个部分组合成完整链接的方法，传入的参数也是一个可迭代对象，例如列表、元组等，唯一的区别是长度必须为5。示例如下：

In [36]:
from urllib.parse import urlunsplit

data = ['http', 'www.baidu.com', 'index.html', 'a=6', 'comment']
print(urlunsplit(data))

http://www.baidu.com/index.html?a=6#comment


### 5. urljoin()

有了urlunparse()和urlunsplit()方法，我们可以完成链接的合并，不过前提必须要有特定长度的对象，链接的每一部分都要清晰分开。

此外，生成链接还有另一个方法，那就是urljoin()方法。我们可以提供一个base_url（基础链接）作为第一个参数，将新的链接作为第二个参数，该方法会分析base_url的scheme、netloc和path这3个内容并对新链接缺失的部分进行补充，最后返回结果。

In [38]:
from urllib.parse import urljoin

print(urljoin('http://www.baidu.com', 'FAQ.html'))
print(urljoin('http://www.baidu.com', 'https://cuiqingcai.com/FAQ.html'))
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html'))
print(urljoin('http://www.baidu.com/about.html', 'https://cuiqingcai.com/FAQ.html?question=2'))
print(urljoin('http://www.baidu.com?wd=abc', 'https://cuiqingcai.com/index.php'))
print(urljoin('http://www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com#comment', '?category=2'))

http://www.baidu.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html
https://cuiqingcai.com/FAQ.html?question=2
https://cuiqingcai.com/index.php
http://www.baidu.com?category=2#comment
www.baidu.com?category=2#comment
www.baidu.com?category=2


### 6. urlencode()

这里我们再介绍一个常用的方法——urlencode()，它在构造GET请求参数的时候非常有用，示例如下：



In [39]:
from urllib.parse import urlencode

params = {
    'name':'germey',
    'age':22
}

base_url = 'http://www.baidu.com?'
url = base_url + urlencode(params)
print(url)

http://www.baidu.com?name=germey&age=22


### 7. parse_qs()

有了序列化，必然就有反序列化。如果我们有一串GET请求参数，利用parse_qs()方法，就可以将它转回字典，示例如下：

In [40]:
from urllib.parse import parse_qs

query = 'name=germey&age=22'
print(parse_qs(query))

{'name': ['germey'], 'age': ['22']}


### 8. parse_qsl()
另外，还有一个parse_qsl()方法，它用于将参数转化为元组组成的列表，示例如下

In [41]:
from urllib.parse import parse_qsl

query = 'name=germey&age=22'
print(parse_qsl(query))

[('name', 'germey'), ('age', '22')]


### 9. quote()

该方法可以将内容转化为URL编码的格式。URL中带有中文参数时，有时可能会导致乱码的问题，此时用这个方法可以将中文字符转化为URL编码，示例如下：

In [43]:
from urllib.parse import quote

keyword = '壁纸'

url = 'https//www.baidu.com/s?wd=' + quote(keyword)
print(url)

https//www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8


### 10. unquote()

有了quote()方法，当然还有unquote()方法，它可以进行URL解码，示例如下：

In [44]:
from urllib.parse import unquote

url = 'https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8'

print(unquote(url))

https://www.baidu.com/s?wd=壁纸


## 3.1.4-分析Robots协议

## 1. Robots协议

Robots协议也称作爬虫协议、机器人协议，它的全名叫作网络爬虫排除标准（Robots Exclusion Protocol），用来告诉爬虫和搜索引擎哪些页面可以抓取，哪些不可以抓取。它通常是一个叫作robots.txt的文本文件，一般放在网站的根目录下。

当搜索爬虫访问一个站点时，它首先会检查这个站点根目录下是否存在robots.txt文件，如果存在，搜索爬虫会根据其中定义的爬取范围来爬取。如果没有找到这个文件，搜索爬虫便会访问所有可直接访问的页面。

该类用起来非常简单，只需要在构造方法里传入robots.txt的链接即可。首先看一下它的声明：

`urllib.robotparser.RobotFileParser(url='')`

当然，也可以在声明时不传入，默认为空，最后再使用set_url()方法设置一下也可。

下面列出了这个类常用的几个方法。

- set_url()：用来设置robots.txt文件的链接。如果在创建RobotFileParser对象时传入了链接，那么就不需要再使用这个方法设置了。
- read()：读取robots.txt文件并进行分析。注意，这个方法执行一个读取和分析操作，如果不调用这个方法，接下来的判断都会为False，所以一定记得调用这个方法。这个方法不会返回任何内容，但是执行了读取操作。
- parse()：用来解析robots.txt文件，传入的参数是robots.txt某些行的内容，它会按照robots.txt的语法规则来分析这些内容。
- can_fetch()：该方法传入两个参数，第一个是User-agent，第二个是要抓取的URL。返回的内容是该搜索引擎是否可以抓取这个URL，返回结果是True或False。
- mtime()：返回的是上次抓取和分析robots.txt的时间，这对于长时间分析和抓取的搜索爬虫是很有必要的，你可能需要定期检查来抓取最新的robots.txt。
- modified()：它同样对长时间分析和抓取的搜索爬虫很有帮助，将当前时间设置为上次抓取和分析robots.txt的时间。

下面我们用实例来看一下：

In [46]:
from urllib.robotparser import RobotFileParser
 
rp = RobotFileParser()
rp.set_url('http://www.jianshu.com/robots.txt')
rp.read()
print(rp.can_fetch('*', 'http://www.jianshu.com/p/b67554025d7d'))
print(rp.can_fetch('*', "http://www.jianshu.com/search?q=python&page=1&type=collections"))

False
False


In [49]:
from urllib.robotparser import RobotFileParser
from urllib.request import urlopen
 
rp = RobotFileParser()
rp.parse(urlopen('http://www.jianshu.com/robots.txt').read().decode('utf-8').split('\n'))
print(rp.can_fetch('*', 'http://www.jianshu.com/p/b67554025d7d'))
print(rp.can_fetch('*', "http://www.jianshu.com/search?q=python&page=1&type=collections"))

HTTPError: HTTP Error 403: Forbidden